# Day 8: Space Image Format
The Elves' spirits are lifted when they realize you have an opportunity to reboot one of their Mars rovers, and so they are curious if you would spend a brief sojourn on Mars. You land your ship near the rover.

When you reach the rover, you discover that it's already in the process of rebooting! It's just waiting for someone to enter a BIOS password. The Elf responsible for the rover takes a picture of the password (your puzzle input) and sends it to you via the Digital Sending Network.

Unfortunately, images sent via the Digital Sending Network aren't encoded with any normal encoding; instead, they're encoded in a special Space Image Format. None of the Elves seem to remember why this is the case. They send you the instructions to decode it.

Images are sent as a series of digits that each represent the color of a single pixel. The digits fill each row of the image left-to-right, then move downward to the next row, filling rows top-to-bottom until every pixel of the image is filled.

Each image actually consists of a series of identically-sized layers that are filled in this way. So, the first digit corresponds to the top-left pixel of the first layer, the second digit corresponds to the pixel to the right of that on the same layer, and so on until the last digit, which corresponds to the bottom-right pixel of the last layer.

For example, given an image 3 pixels wide and 2 pixels tall, the image data `123456789012` corresponds to the following image layers:

```
Layer 1: 123
         456

Layer 2: 789
         012

```

The image you received is 25 pixels wide and 6 pixels tall.

To make sure the image wasn't corrupted during transmission, the Elves would like you to find the layer that contains the fewest 0 digits. On that layer, what is the number of 1 digits multiplied by the number of 2 digits?

In [1]:
import tools

In [3]:
data = tools.get_data(8)

In [5]:
num_pixels = 25 * 6

In [8]:
num_layers = len(data.strip()) // num_pixels

In [9]:
num_layers

100

In [12]:
import itertools

def make_image(data, shape):
    x, y = shape
    n_layers = len(data.strip()) // (x * y)
    data_iter = (int(c) for c in data.strip())
    return [[[next(data_iter) for _ in range(x)] for _ in range(y)] for _ in range(n_layers)]

In [14]:
image = make_image(data, (25, 6))

In [15]:
layer_with_fewest_0s = min(image, key=lambda layer: sum(1 for row in layer for val in row if val == 0))

In [16]:
def count_digits_in_layer(layer, digit):
    return sum(1 for row in layer for val in row if val == digit)

In [18]:
count_digits_in_layer(layer_with_fewest_0s, 1) * count_digits_in_layer(layer_with_fewest_0s, 2)

1862

### Part Two
Now you're ready to decode the image. The image is rendered by stacking the layers and aligning the pixels with the same positions in each layer. The digits indicate the color of the corresponding pixel: 0 is black, 1 is white, and 2 is transparent.

The layers are rendered with the first layer in front and the last layer in back. So, if a given position has a transparent pixel in the first and second layers, a black pixel in the third layer, and a white pixel in the fourth layer, the final image would have a black pixel at that position.

For example, given an image 2 pixels wide and 2 pixels tall, the image data `222112222120000`corresponds to the following image layers:

```
Layer 1: 02
         22

Layer 2: 11
         22

Layer 3: 22
         12

Layer 4: 00
         00
```      

Then, the full image can be found by determining the top visible pixel in each position:

- The top-left pixel is black because the top layer is 0.
- The top-right pixel is white because the top layer is 2 (transparent), but the second layer is 1.
- The bottom-left pixel is white because the top two layers are 2, but the third layer is 1.
- The bottom-right pixel is black because the only visible pixel in that position is 0 (from layer 4).
So, the final image looks like this:
```
01
10
```

What message is produced after decoding your image?

In [27]:
def shape(layer):
    return len(layer[0]), len(layer)

In [59]:
def flatten_image(layers):
    x, y = shape(layers[0])
    image = [[2] * x] * y
    for i in range(x):
        for j in range(y):
            image[j][i] = next((layer[j][i] for layer in layers if layer[j][i] != 2), 2)
    return image

In [48]:
flattened = flatten_image(image)

In [49]:
flattened

[[0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0]]

In [32]:
def display_image(image):
    flattened = flatten_image(image)
    for row in flattened:
        print(''.join(' X-'[channel] for channel in row))

In [33]:
display_image(image)

 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 


In [35]:
def bit_layers(image, index):
    x, y = index
    return [layer[y][x] for layer in image]

In [37]:
def resolve_transparency(bit_layer):
    return next((bit for bit in bit_layer if bit != 2), 2)

In [38]:
def decode_bit(image, index):
    return resolve_transparency(bit_layers(image, index))

In [39]:
decode_bit(image, (0, 0))

0

In [40]:
[decode_bit(image, (x, 0)) for x in range(25)]

[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]

In [41]:
def decode_image(image):
    x, y = shape(image[0])
    return [[decode_bit(image, (i, j)) for i in range(x)] for j in range(y)]

In [42]:
decode_image(image)

[[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0]]

In [44]:
def display_image_decode(image):
    decoded = decode_image(image)
    for row in decoded:
        print(''.join(' X-'[channel] for channel in row))

In [46]:
display_image_decode(image)

 XX   XX  XXX  X  X X    
X  X X  X X  X X  X X    
X    X    X  X XXXX X    
X XX X    XXX  X  X X    
X  X X  X X    X  X X    
 XXX  XX  X    X  X XXXX 


In [50]:
def test_get_bit(i, j):
    return next((layer[j][i] for layer in image if layer[j][i] != 2), 2)

In [55]:
test_get_bit(0, 1)

1

In [60]:
display_image(image)

 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 
 XXX  XX  X    X  X XXXX 


Notice that this corresponds to the last row of the actual output.

This is because I used `image = [[2] * x] * y` in `flatten_image`, which is fine for a single row, but actually causes all rows to be the same array object, so earlier "rows" get overwritten by subsequent rows.